# Use small manually-labeled set as examples and try few-shot of GPT model.

In [36]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
import pandas as pd
from langchain import PromptTemplate, OpenAI, LLMChain

In [53]:
import re

In [10]:
from config import *
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
sample_label_path = 'data/labelled_sample.csv'

In [7]:
df_sample = pd.read_csv(sample_label_path, encoding="GBK")

In [8]:
df_sample.head()

,CONTENT,TOPIC
0,推进基础设施建设和区域协调发展。京津冀协同发展、长江经济带建设取得重要进展。新建铁路投产里程...,基建
1,七是加快建立多主体供应、多渠道保障、租购并举的住房制度。要发展住房租赁市场特别是长期租赁，保...,房地产
2,会议认为，坚持标准引领，建设制造强国，是结构性改革尤其是供给侧结构性改革的重要内容，有利于改...,制造业
3,会议指出，党的十八大以来，党中央部署推进科技评价体系改革，聚焦“四个面向”的科技成果评价导向...,科技
4,会议强调，完善促进消费体制机制，要顺应居民消费新趋势，从供需两端发力，积极培育重点消费领域细...,零售消费


In [41]:
df_sample.shape

(24, 2)

In [27]:
examples = [{'content':row['CONTENT'],'topic':row['TOPIC']} for index, row in df_sample.iterrows()]

In [28]:
examples[:3]

[{'content': '推进基础设施建设和区域协调发展。京津冀协同发展、长江经济带建设取得重要进展。新建铁路投产里程8427公里，高速铁路运营里程达1.6万公里，占世界的60%以上。高速公路通车里程达11.2万公里，水路、民航、管道建设进一步加强。农网改造稳步进行。宽带用户超过7.8亿户。经过多年努力，南水北调中线一期工程正式通水，惠及沿线亿万群众。',
  'topic': '基建'},
 {'content': '七是加快建立多主体供应、多渠道保障、租购并举的住房制度。要发展住房租赁市场特别是长期租赁，保护租赁利益相关方合法权益，支持专业化、机构化住房租赁企业发展。完善促进房地产市场平稳健康发展的长效机制，保持房地产市场调控政策连续性和稳定性，分清中央和地方事权，实行差别化调控。',
  'topic': '房地产'},
 {'content': '会议认为，坚持标准引领，建设制造强国，是结构性改革尤其是供给侧结构性改革的重要内容，有利于改善供给、扩大需求，促进产品产业迈向中高端。会议通过了《装备制造业标准化和质量提升规划》，要求对接《中国制造2025》，瞄准国际先进水平，实施工业基础和智能制造、绿色制造标准化和质量提升工程，加快关键技术标准研制，推动在机器人、先进轨道交通装备、农业机械、高性能医疗器械等重点领域标准化实现新突破，并适应创新进展和市场需求及时更新标准，力争到2020年使重点领域国际标准转化率从目前的70%以上提高到90%以上。要弘扬工匠精神，追求精益求精，生产更多有创意、品质优、受群众欢迎的产品，坚决淘汰不达标产品，提振消费者对“中国制造”的信心，支撑制造业提质增效、提升国际竞争力。',
  'topic': '制造业'}]

In [42]:
pre_fix_template="""你需要给一段文字打上话题标签。我会给你话题列表和文字内容。话题必须从给定的话题列表里选出，而且只能给一个话题。
话题列表是：{topic_list}。
你的回答的格式必须是：回答-话题
请给出这段文字的话题。你的回答仅包括话题。你给出的话题必须是话题列表里面的一个，而且只能给一个话题。""".format(topic_list = ','.join(topics_pdf))

In [43]:
print(pre_fix_template)

你需要给一段文字打上话题标签。我会给你话题列表和文字内容。话题必须从给定的话题列表里选出。
话题列表是：基建,房地产,制造业,科技,零售消费,绿色发展,医药健康,教育,交通运输,出口贸易,农业农村,文化旅游,就业,民生扶贫,能源,物资储备,宏观,区域发展,企业发展,新冠疫情,地方财政,税务,国际关系,金融。
你的回答的格式必须是：回答-话题
请给出这段文字的话题。你的回答仅包括话题。你给出的话题必须是话题列表里面的一个。


In [44]:
example_prompt = PromptTemplate(input_variables=["content", "topic"], template="文字内容： {content}\n回答-话题：{topic}")

In [35]:
prompt = FewShotPromptTemplate(
    examples=examples[:5], 
    example_prompt=example_prompt, 
    prefix=pre_fix_template,
    suffix="文字内容： {content}", 
    input_variables=['content']
)

print(prompt.format(content=examples[11]['content']))


你需要给一段文字打上话题标签。我会给你话题列表和文字内容。话题必须从给定的话题列表里选出。
话题列表是：基建,房地产,制造业,科技,零售消费,绿色发展,医药健康,教育,交通运输,出口贸易,农业农村,文化旅游,就业,民生扶贫,能源,物资储备,宏观,区域发展,企业发展,新冠疫情,地方财政,税务,国际关系,金融。
你的回答的格式必须是：回答-话题
请给出这段文字的话题。你的回答仅包括话题。你给出的话题必须是话题列表里面的一个。


文字内容： 推进基础设施建设和区域协调发展。京津冀协同发展、长江经济带建设取得重要进展。新建铁路投产里程8427公里，高速铁路运营里程达1.6万公里，占世界的60%以上。高速公路通车里程达11.2万公里，水路、民航、管道建设进一步加强。农网改造稳步进行。宽带用户超过7.8亿户。经过多年努力，南水北调中线一期工程正式通水，惠及沿线亿万群众。
回答-话题：基建

文字内容： 七是加快建立多主体供应、多渠道保障、租购并举的住房制度。要发展住房租赁市场特别是长期租赁，保护租赁利益相关方合法权益，支持专业化、机构化住房租赁企业发展。完善促进房地产市场平稳健康发展的长效机制，保持房地产市场调控政策连续性和稳定性，分清中央和地方事权，实行差别化调控。
回答-话题：房地产

文字内容： 会议认为，坚持标准引领，建设制造强国，是结构性改革尤其是供给侧结构性改革的重要内容，有利于改善供给、扩大需求，促进产品产业迈向中高端。会议通过了《装备制造业标准化和质量提升规划》，要求对接《中国制造2025》，瞄准国际先进水平，实施工业基础和智能制造、绿色制造标准化和质量提升工程，加快关键技术标准研制，推动在机器人、先进轨道交通装备、农业机械、高性能医疗器械等重点领域标准化实现新突破，并适应创新进展和市场需求及时更新标准，力争到2020年使重点领域国际标准转化率从目前的70%以上提高到90%以上。要弘扬工匠精神，追求精益求精，生产更多有创意、品质优、受群众欢迎的产品，坚决淘汰不达标产品，提振消费者对“中国制造”的信心，支撑制造业提质增效、提升国际竞争力。
回答-话题：制造业

文字内容： 会议指出，党的十八大以来，党中央部署推进科技评价体系改革，聚焦“四个面向”的科技成果评价导向逐步确立，激励创新的环境正在形成，带动我国科技创新能力明显提升。同时，分类的科技成果评价体系尚未建

In [32]:
examples[11]

{'content': '丰富人民群众精神文化生活。培育和践行社会主义核心价值观。深化群众性精神文明创建。发展新闻出版、广播影视、文学艺术、哲学社会科学和档案等事业，加强智库建设。扎实推进媒体深度融合。提升国际传播效能。加强和创新互联网内容建设。弘扬中华优秀传统文化，加强文物和文化遗产保护传承。实施文化惠民工程，公共图书馆、博物馆、美术馆、文化馆站向社会免费开放。深入推进全民阅读。支持文化产业发展。加强国家科普能力建设。体育健儿勇创佳绩，全民健身广泛开展。',
 'topic': '文化旅游'}

In [38]:
llm = OpenAI(temperature = 0, openai_api_key = OPENAI_API_KEY, model_name = 'gpt-4')

F:\anaconda3\lib\site-packages\langchain\llms\openai.py:769: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [39]:
llm_chain = LLMChain(
    llm = llm,
    prompt = prompt
)

In [40]:
llm_chain.predict(content=examples[11]['content'])

'回答-话题：文化旅游'

In [50]:
df_sample_para = pd.read_csv('data/sample.csv', encoding="utf_8_sig")

In [51]:
df_sample_para.head()

,ID,RAW_CONTENT,MEETING_NAME,TITLE,PUBLISH_DATE
0,m-941334512-1560944340,李克强主持召开国务院常务会议\r\n部署推进城镇老旧小区改造 顺应群众期盼改善居住条件\r\...,国务院常委会,李克强主持召开国务院常务会议部署推进城镇老旧小区改造 顺应群众期盼改善居住条件确定提前完成农...,2019-06-19T19:39:00Z
1,m-221628530-1512033180,\r\n 中共中央政治局11月30日召开会议，审议通过《中国共产党党务公开条例（试行）》。...,政治局会议,审议通过中国共产党党务公开条例试行,2017-11-30T17:13:00Z
2,m-2186953670-1504012260,中共中央总书记、国家主席、中央军委主席、中央全面深化改革领导小组组长习近平8月29日下午...,中央全面深化改革会议,会议审议通过了关于完善主体功能区战略和制度的若干意见关于探索建立涉农资金统筹整合长效机制的意...,2017-08-29T21:11:00Z
3,m-1244349627-1621601580,中共中央总书记、国家主席、中央军委主席、中央全面深化改革委员会主任习近平5月21日下午主...,中央全面深化改革会议,审议通过了关于完善科技成果评价机制的指导意见关于进一步减轻义务教育阶段学生作业负担和校外培训...,2021-05-21T20:53:00Z
4,m-465612500-1661856840,中共中央政治局8月30日召开会议，研究中国共产党第十九届中央委员会第七次全体会议和中国共...,政治局会议,研究中国共产党第十九届中央委员会第七次全体会议和中国共产党第二十次全国代表大会筹备工作,2022-08-30T18:54:00Z


In [52]:
df_sample_para.shape

(53, 5)

In [54]:
id_list = []
title_list = []
content_list = []
topic_list = []
ln_min = 50  
cnt = 0

In [57]:
for index, row in df_sample_para.iterrows():
    txt = re.split('\u3000\u3000|\r\n', row['RAW_CONTENT'])
    txt = [x.strip() for x in txt]
    for x in txt:
        if len(x) >= ln_min:
            id_list.append(row['ID'])
            title_list.append(row['TITLE'])
            content_list.append(x)
            topic_list.append(llm_chain.predict(content=x).split('话题：')[-1])
            cnt += 1
            if cnt % 100 == 0:
                print(str(cnt) + ' paragraphs have been processed...')

100 paragraphs have been processed...
200 paragraphs have been processed...
300 paragraphs have been processed...
400 paragraphs have been processed...


In [58]:
df_sample_para_few = pd.DataFrame({'ID': id_list, 
                               'TITLE': title_list,
                               'CONTENT': content_list,
                              'TOPIC': topic_list})

In [59]:
df_sample_para_few.head()

,ID,TITLE,CONTENT,TOPIC
0,m-941334512-1560944340,李克强主持召开国务院常务会议部署推进城镇老旧小区改造 顺应群众期盼改善居住条件确定提前完成农...,国务院总理李克强6月19日主持召开国务院常务会议，部署推进城镇老旧小区改造，顺应群众期盼改善...,民生扶贫
1,m-941334512-1560944340,李克强主持召开国务院常务会议部署推进城镇老旧小区改造 顺应群众期盼改善居住条件确定提前完成农...,会议指出，按照中央经济工作会议和《政府工作报告》部署，积极做好“六稳”工作，稳投资是重要方面...,基建
2,m-941334512-1560944340,李克强主持召开国务院常务会议部署推进城镇老旧小区改造 顺应群众期盼改善居住条件确定提前完成农...,会议认为，加快改造城镇老旧小区，群众愿望强烈，是重大民生工程和发展工程。据各地初步摸查，目前...,民生扶贫
3,m-941334512-1560944340,李克强主持召开国务院常务会议部署推进城镇老旧小区改造 顺应群众期盼改善居住条件确定提前完成农...,会议指出，新一轮农村电网改造升级工程自2016年实施以来，完成了农村机井通电、贫困村通动力电...,农业农村
4,m-941334512-1560944340,李克强主持召开国务院常务会议部署推进城镇老旧小区改造 顺应群众期盼改善居住条件确定提前完成农...,会议指出，通过实施农村供水工程，农村居民饮水已提前实现联合国千年发展目标，农村集中供水率、自...,农业农村


In [60]:
df_sample_para_few['TOPIC'].value_counts()

宏观                      92
民生扶贫                    51
企业发展                    49
教育                      45
绿色发展                    29
出口贸易                    29
农业农村                    26
医药健康                    22
国际关系                    21
区域发展                    14
金融                      14
新冠疫情                    13
基建                      13
地方财政                    12
税务                       9
科技                       9
政治                       7
就业                       7
文化旅游                     6
零售消费                     5
制造业                      3
安全生产                     2
国防                       2
物资储备                     2
政府工作                     2
军事                       2
政府管理                     1
政策                       1
安全监管                     1
房地产                      1
科技,教育,医药健康,绿色发展,文化旅游     1
能源                       1
法律                       1
政府职能                     1
企业发展,税务                  1
教育,医药健康,绿色发展,文化旅游,科技     1
政府决策                     1
N

In [61]:
df_sample_para_few.to_csv('data/sample_para_few.csv', index = False, encoding='utf_8_sig')

In [62]:
df_sample_para_few.shape

(497, 4)